# explorerNapari
> Purpose: Navigate data and maybe save screenshots for presentations and figures

- [x] Requires: [NAPARI](https://napari.org/) and [cellpose](https://cellpose-napari.readthedocs.io/en/latest/index.html)

In [ ]:
## to install
# !pip install napari[all]
# !pip install QT
# !pip install napari[pyqt5]
# !pip install cellpose-napari==0.1.4
# !pip install napari-nikon-nd2
# !pip install magicgui
# !pip install napari-pyclesperanto-assistant
# !pip install napari-clusters-plotter #hdbscan failed
# !pip install napari-plot-profile
# !pip install napari-brightness-contrast
# !pip install napari-curtain
# !pip install napari-3d-ortho-viewer
# !pip install napari-manual-split-and-merge-labels
# !pip install napari-crop
# !pip install napari-stl-exporter

# !conda install -c conda-forge ipywidgets

## to upgrade:
# !pip install cellpose-napari==0.1.4
# !pip install cellpose-napari --upgrade
# !pip install cellpose --upgrade
# !pip install napari --upgrade
# !pip install mxnet-mkl

In [ ]:
import napari
from magicgui import magicgui
from enum import Enum
import cellpose_napari
import cellpose
from cellpose import models
from napari.utils import nbscreenshot
from tifffile import imread
import numpy as np
from scipy import ndimage
from scipy.stats import mannwhitneyu
import napari_nikon_nd2
import os 
from magicgui import magicgui
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import importlib
import svgutils
from svgutils.compose import *
import ipywidgets
from tqdm import tqdm
import time
import sys
from plotParams import *
from IPython.lib.display import Audio


pBGMenu = ipywidgets.Dropdown(options=['Light', 'Dark'],value='Dark',disabled=False, layout=ipywidgets.Layout(width='20%', height='40px'))
pBGLabel = ipywidgets.widgets.Label('Select plotting style', layout=ipywidgets.Layout(width='20%', height='30px'))
pBG = ipywidgets.widgets.VBox([pBGLabel,pBGMenu])
pBG

In [ ]:
applyPlotStyle(pBGMenu.value)
viewer = napari.Viewer()

In [ ]:
# viewer.close()

***
# Index <a id='Index'></a>
***
- [Define directory paths](#pathNames)
- [exploreImages](#exploreImages)

***
# Define pathnames<a id='pathNames' name='pathNames'>∮</a>
***
[Back to Index](#Index)

#### For list of filenanames:

```text
gene = 'nr2e3'
fList = !ls $dPath/*$gene*[002,004].nd2
fList = list(map(lambda st: str.replace(st, dPath + '/', 'fPath = \"'), fList))
fList = list(map(lambda st: str.replace(st, '.nd2', '\"; gene = \'' + gene + '\'; '), fList))
print(*fList, sep = "\n")
```

In [ ]:
# clear viewer
viewer.layers.select_all(); viewer.layers.remove_selected()
# data directory
dPath = "/Volumes/angueyra04/LiImaging/20220923_tbx2aF0s_aC3/"
# analysis directory
dAnalysis = "/Users/angueyraaristjm/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/zf/Analysis/CRtbx2F0s/activatedCaspase3/"

In [ ]:
# get file names
gene = 'wt'
fList = !ls $dPath/*$gene*[001,003].nd2
fList = list(map(lambda st: str.replace(st, dPath + '/', 'fPath = \"'), fList))
fList = list(map(lambda st: str.replace(st, '.nd2', '\"; gene = \'' + gene + '\'; '), fList))
print(*fList, sep = "\n")

***
# Explore Images<a id='exploreImages' name='exploreImages'>∮</a>
***
[Back to Index](#Index)

In [ ]:
# 5dpf

# open file
dPath = "/Volumes/angueyra04/LiImaging/20220914_tbx2aF0s_aC3/"

# whole-eyes
# fPath = "20220914_5dpf_xOG_s1C_aCaspase3r647_wt_L01_001"; gene = 'wt'; #saturated; can't mip
# fPath = "20220914_5dpf_xOG_s1C_aCaspase3r647_wt_L01_003"; gene = 'wt'; # ok; 1 cell in retina
# fPath = "20220914_5dpf_xOG_s1C_aCaspase3r647_wt_L02_001"; gene = 'wt'; # damaged
# fPath = "20220914_5dpf_xOG_s1C_aCaspase3r647_wt_L02_003"; gene = 'wt'; # no positive cells here
# fPath = "20220914_5dpf_xOG_s1C_aCaspase3r647_wt_L03_001"; gene = 'wt'; # no positive cells here
# fPath = "20220914_5dpf_xOG_s1C_aCaspase3r647_wt_L03_003"; gene = 'wt'; # no positive cells here
# fPath = "20220914_5dpf_xOG_s1C_aCaspase3r647_wt_L04_001"; gene = 'wt'; 
# fPath = "20220914_5dpf_xOG_s1C_aCaspase3r647_wt_L04_003"; gene = 'wt'; # this is EXAMPLE
# fPath = "20220914_5dpf_xOG_s1C_aCaspase3r647_wt_L05_001"; gene = 'wt'; 
# fPath = "20220914_5dpf_xOG_s1C_aCaspase3r647_wt_L05_003"; gene = 'wt'; 
# fPath = "20220914_5dpf_xOG_s1C_aCaspase3r647_wt_L06_001"; gene = 'wt'; 
# fPath = "20220914_5dpf_xOG_s1C_aCaspase3r647_wt_L06_003"; gene = 'wt'; 

# fPath = "20220914_5dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L01_001"; gene = 'tbx2a'; #quite dim; some positive cells in CMZ (deep in stack)
# fPath = "20220914_5dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L01_003"; gene = 'tbx2a'; #quite dim, but some cells in inner retina
# fPath = "20220914_5dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L02_001"; gene = 'tbx2a'; # one positive cell in inner retina, some more in CMZ; could be good EXAMPLE to wt L004
# fPath = "20220914_5dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L02_003"; gene = 'tbx2a'; # damaged; again not a lot of positive cells
# fPath = "20220914_5dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L03_001"; gene = 'tbx2a'; # very dim cells
# fPath = "20220914_5dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L03_003"; gene = 'tbx2a'; # better signal but some damage
# fPath = "20220914_5dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L04_001"; gene = 'tbx2a'; # no visible cells
# fPath = "20220914_5dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L04_003"; gene = 'tbx2a'; # no positive cells; this one clearly shows to UV cones in SZ. Could be used as example
# fPath = "20220914_5dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L05_001"; gene = 'tbx2a'; # damaged dorsal retina
# fPath = "20220914_5dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L05_003"; gene = 'tbx2a'; # could be saveable for a few slices. Not sure it's nalou
# fPath = "20220914_5dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L06_001"; gene = 'tbx2a'; # promising but very noisy stack
# fPath = "20220914_5dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L06_003"; gene = 'tbx2a'; # not bad

# zoom ins
fPath = "20220914_5dpf_xOG_s1C_aCaspase3r647_wt_L04_004"; gene = 'wt'; # this is EXAMPLE
fPath = "20220914_5dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L06_004"; gene = 'tbx2a'; # one positive cell in inner retina, some more in CMZ; could be good EXAMPLE to wt L004



In [ ]:
# clear viewer
viewer.layers.select_all(); viewer.layers.remove_selected()

# 3dpf

# open file
dPath = "/Volumes/angueyra04/LiImaging/20220923_tbx2aF0s_aC3/"

# whole-eyes
# # fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_wt_L01_001"; gene = 'wt'; # cones up
# fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_wt_L01_003"; gene = 'wt'; # some damage
# # fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_wt_L02_001"; gene = 'wt'; # cones up
# fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_wt_L02_003"; gene = 'wt'; # ok
# # fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_wt_L03_001"; gene = 'wt'; # cones up
# fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_wt_L03_003"; gene = 'wt'; #DAPI too bright
# # fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_wt_L04_001"; gene = 'wt'; # cones up
# # fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_wt_L04_003"; gene = 'wt'; # cones up
# fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_wt_L05_001"; gene = 'wt'; # EXAMPLE
# # fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_wt_L05_003"; gene = 'wt'; # cones up
# # fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_wt_L06_001"; gene = 'wt'; # cones up
# fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_wt_L06_003"; gene = 'wt'; # nice one too

# these feel younger; skipping for now
# fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_wt_L07_001"; gene = 'wt'; # cones up
# fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_wt_L07_003"; gene = 'wt'; # cones up
# fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_wt_L08_001"; gene = 'wt'; # cones up
# fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_wt_L08_003"; gene = 'wt'; 
# fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_wt_L09_001"; gene = 'wt'; 
# fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_wt_L09_003"; gene = 'wt'; 
# fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_wt_L10_001"; gene = 'wt'; 
# fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_wt_L10_003"; gene = 'wt'; 
# fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_wt_L11_001"; gene = 'wt'; 
# fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_wt_L11_003"; gene = 'wt'; 
# fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_wt_L12_001"; gene = 'wt'; 

# fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L01_001"; gene = 'tbx2a'; # cones up
fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L01_003"; gene = 'tbx2a'; # ok
# fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L02_001"; gene = 'tbx2a'; # cones up
fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L02_003"; gene = 'tbx2a'; # ok too
# fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L03_001"; gene = 'tbx2a'; # cones up
fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L03_003"; gene = 'tbx2a'; 
# fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L04_001"; gene = 'tbx2a'; # cones up
# fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L04_003"; gene = 'tbx2a'; # cones up
# fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L05_001"; gene = 'tbx2a'; # cones up
fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L05_003"; gene = 'tbx2a'; 
# fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L06_001"; gene = 'tbx2a'; # cones up
fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L06_003"; gene = 'tbx2a'; 
# fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L07_001"; gene = 'tbx2a'; # cones up
fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L07_003"; gene = 'tbx2a'; 
# fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L08_001"; gene = 'tbx2a'; # cones up
fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L08_003"; gene = 'tbx2a'; 
# fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L09_001"; gene = 'tbx2a'; # cones up
fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L09_003"; gene = 'tbx2a'; 
# fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L10_001"; gene = 'tbx2a'; # cones up
fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L10_003"; gene = 'tbx2a'; 
# fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L11_001"; gene = 'tbx2a'; # cones up; damaged

# zoom ins
fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_wt_L06_004"; gene = 'wt'; # nice one too
fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_wt_L05_002"; gene = 'wt'; # EXAMPLE
fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L09_004"; gene = 'tbx2a';
fPath = "20220923_3dpf_xOG_s1C_aCaspase3r647_F0tbx2a_L10_004"; gene = 'tbx2a';

# load whole stack to figure out best planes for cellpose
viewer.open((dPath+fPath + ".nd2"))
viewer.layers.select_next();
if len(viewer.layers)==2: # no DAPI, no Cy5, no DIC
    nChannels = 2
    viewer.layers[0].colormap = 'green'
    viewer.layers[0].name = 'R'
    viewer.layers[1].colormap = 'magenta'
    viewer.layers[1].name = 'U'
elif len(viewer.layers)==3: # no DAPI, no Cy5
    nChannels = 3
    viewer.layers[0].colormap = 'green'
    viewer.layers[0].name = 'R'
    viewer.layers[1].colormap = 'magenta'
    viewer.layers[1].name = 'U'
    viewer.layers[2].colormap = 'gray'
    viewer.layers[2].name = 'DIC'
elif len(viewer.layers)==4: # no DAPI
    nChannels = 4
    viewer.layers[0].colormap = 'green'
    viewer.layers[0].name = 'R'
    viewer.layers[1].colormap = 'magenta'
    viewer.layers[1].name = 'U'
    viewer.layers[2].colormap = 'yellow'
    viewer.layers[2].name = 'aC3'
    viewer.layers[3].colormap = 'gray'
    viewer.layers[3].name = 'DIC'
elif len(viewer.layers)==5:
    nChannels = 5
    viewer.layers[0].colormap = 'gray'
    viewer.layers[0].name = 'N'
    viewer.layers[0].opacity = 0.33
    viewer.layers[1].colormap = 'green'
    viewer.layers[1].name = 'R'
    viewer.layers[2].colormap = 'magenta'
    viewer.layers[2].name = 'U'
    viewer.layers[3].colormap = 'yellow'
    viewer.layers[3].name = 'aC3'
    viewer.layers[4].colormap = 'gray'
    viewer.layers[4].name = 'DIC'

    
print('Loaded!')
display(Audio('/Users/angueyraaristjm/PythonCode/RNAseq/OOT_Fanfare_Item.wav', autoplay=True))


In [ ]:
# make mips (and remove old ones)
zlims = [1,70];

if len(viewer.layers)>nChannels:
    for l in viewer.layers[nChannels:]:
        viewer.layers.remove(l)

for l in viewer.layers[0:nChannels]:
    l.visible = False
    viewer.add_image(l.data[zlims[0]:zlims[1]].max(axis=0), blending='additive', colormap = l.colormap, name = l.name + "_mip")

In [ ]:
# create subdirectory for analysis
dOut = dAnalysis + gene + '/' + fPath + '/'
if (os.path.isdir(dOut)==False):
    os.mkdir(dOut)
    print('Created output directory')

    
# when things look good, save in folder for batch cellpose analysis
l = viewer.layers['R_mip']; l.save(dOut + l.name + '.tiff')
l = viewer.layers['U_mip']; l.save(dOut + l.name + '.tiff')
l = viewer.layers['N_mip']; l.save(dOut + l.name + '.tiff')
l = viewer.layers['aC3_mip']; l.save(dOut + l.name + '.tiff')
l = viewer.layers['DIC_mip']; l.save(dOut + l.name + '.tiff')
print('MIP layers saved for ' + fPath)
# l = viewer.layers['LWS_mip']; l.save(dOut + l.name + '.tiff')

### [Go back and run next one &uarr;](#zExtract)
---

### Read csv created during analysis and create bar plot

In [ ]:
#gene Colors
zfC = {
    'R'  : '#7d7d7d',
    'U' : '#B73AB9',
    'S' : '#4364F6',
    'M' : '#59CB3B',
    'L' : '#CE2A22',
}

zfG = {
    'wt' : '#000000',
    'tbx2a' : '#ab266b',
    'tbx2b' : '#421f8e',
    'foxq2' : '#001dd6',
    'nr2e3' : '#7d7d7d',
}

zfGm = {
    'wt' : 'o',
    'tbx2a' : 'P',
    'tbx2b' : 'X',
    'nr2e3' : '+',
    
}

prLabel = {
    'R'  : 'Rods',
    'U' : 'UV',
    'S' : 'S',
    'M' : 'M',
    'L' : 'L',
}


def formatFigureMain(figH, axH, plotH):
#     font_path = 'C:/Users/pataklk/Documents/Frag_Analysis_Code/Avenir.ttc'
    font_path = '/System/Library/Fonts/Avenir.ttc'
    fontTicks = font_manager.FontProperties(fname=font_path, size=30) # was 18
    fontLabels = font_manager.FontProperties(fname=font_path, size=36) # was 22
    fontTitle = font_manager.FontProperties(fname=font_path, size=30) # was 28 
    axH.set_xscale('linear')
    axH.spines['top'].set_visible(False)
    axH.spines['right'].set_visible(False)
    
    for label in (axH.get_xticklabels() + axH.get_yticklabels()):
        label.set_fontproperties(fontTicks)
    axH.set_xlabel(axH.get_xlabel(), fontproperties = fontTicks)
    axH.set_ylabel(axH.get_ylabel(), fontproperties = fontTicks)
    return fontLabels

def formatFigure(figH, axH, plotH):
    fontLabels = formatFigureMain(figH, axH, plotH)
#     axH.set_xlabel('wt vs. cr', fontproperties=fontLabels)
    axH.set_ylabel('cells per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.xaxis.set_tick_params(rotation=45)

def formatFigureRvU(figH, axH, plotH):
    fontLabels = formatFigureMain(figH, axH, plotH)
    axH.set_xlabel('Rods per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.set_ylabel('UV cones per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.xaxis.set_tick_params(rotation=45)
    
def formatFigureMvS(figH, axH, plotH):
    fontLabels = formatFigureMain(figH, axH, plotH)
    axH.set_xlabel('M cones per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.set_ylabel('S cones per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.xaxis.set_tick_params(rotation=45)

    
def lighten_color(color, amount=0.5):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    import colorsys
    try:
        c = matplotlib.colors.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*matplotlib.colors.to_rgb(c))
    return matplotlib.colors.rgb2hex(colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2]))

def estimateJitter(dataArray):
    """ creates random jitter scaled by local density of points"""
    from scipy.stats import gaussian_kde
    kde = gaussian_kde(dataArray)
    density = kde(dataArray)
    jitter = np.random.randn(len(dataArray))*density
    return jitter
pBGMenu

In [ ]:
#reapply plotting style
applyPlotStyle(pBGMenu.value)

In [ ]:
# Created csv manually during analysis
dPath = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/CRnr2e3F0s/xOGaCT/"
fName = "CRnr2e3F0s_xOGaCT_Counts.csv"

dPath = "/Users/angueyraaristjm/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/zf/Analysis/CRnr2e3F0s/"
# dPath = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/CRnr2e3F0s/m2Gs2C/"
fName = "CRnr2e3F0s_combinedCounts.csv"

dPath = "/Users/angueyraaristjm/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/zf/Analysis/CRtbx2F0s/"
fName = "CRtbx2F0s_combinedCounts.csv"

df = pd.read_csv(dPath + fName)
df

In [ ]:
geneList = ['wt','tbx2a', 'tbx2b']
nGenes = np.size(geneList)
photoreceptors = ['R','U','S','M','L']
# photoreceptors = ['S','M']

plotname = ''
fH, axH = plt.subplots(figsize= [12,8])

barW = nGenes+2; # bar width
barD = nGenes+1; # bar distance whitin 1 photoreceptor subtype
barP = -nGenes+1; # position in x-axis
barStep = 0.9; # distance between photoreceptor groups
barPos = []; #array to save bar positions


j=barP;
for gene in geneList:
    i=0;
    j=j+1;
    for pr in photoreceptors:
        i = i+barStep;
        countData = df[(df['genotype']==gene) & (df['excludeFlag']==False)][pr]
        countData = countData[~np.isnan(countData)]
        jitter = np.random.randn(len(countData))*0.03
#         textindent = 1/(2.5*barD);
        textindent = 0.10;
        pH = plt.bar([i+j/barD], np.mean(countData), width=1/barW, color=lighten_color(zfC[pr],1), linewidth = 2, edgecolor = lighten_color(zfC[pr],1)); #color=zfC[pr], 
#         pH = plt.bar([i+j/barD], np.mean(countData), yerr = sem(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr][~np.isnan(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr])], ddof = 0), align='center', ecolor='black', capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
#         pH = plt.bar([i+j/barD], np.mean(countData), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr][~np.isnan(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr])], ddof = 0), align='center', ecolor='black', capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
       # pH = plt.bar([i+j/barD], np.mean(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), ecolor=lighten_color(zfC[pr],1.2), capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
        #pH = plt.errorbar([i+j/barD], np.mean(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), ecolor=lighten_color(zfC[pr],1.1), elinewidth=2, capsize=5, capthick=2, barsabove= True)
#         pH = plt.scatter(np.ones(len(countData))*[i+j/barD]+jitter, countData, color=lighten_color(zfC[pr],2/3), zorder=2, marker = zfGm[gene], edgecolor='#ffffff', linewidth=0.5, alpha = 0.4);
        pH = plt.errorbar(i+j/barD, np.mean(countData), yerr = [[0],[np.std(countData)]], ecolor=lighten_color(zfC[pr],1), elinewidth=3, capsize=8, capthick=3, zorder=9)
        # plt.text((i+j/barD)-textindent, 3, prLabel[pr], font_properties=font_prop, fontsize=20, ha='left', alpha=0.6)
        pH = plt.scatter(np.ones(len(countData))*[i+j/barD]+jitter, countData, color='#ffffff', zorder=8, marker = 'o', s=50, edgecolor='#ffffff', linewidth=0.5, alpha = .6);
        barPos = np.append(barPos,(i+j/barD))

formatFigure(fH, axH, pH)
axH.set_xticks(np.sort(barPos));

axH.set_xticklabels(geneList * len(photoreceptors));
# axH.set_ylim([0,500]); # this was 400

# savePath = 'C:/Users/pataklk/OneDrive - National Institutes of Health/zf/F0_Analysis/CRfoxq2F0/'
# savePath = "/Users/angueyraaristjm/OneDrive - National Institutes of Health/zf/F0_Analysis/CRfoxq2F0/"
# savePath = "/Users/angueyraaristjm/Documents/LiLab/Presentations/revealjs/resources/20211008_UCLA/"
# plt.savefig(savePath + "Counts_foxq2.png", transparent=True, format="png", bbox_inches = "tight")
# plt.savefig(savePath + "Counts_foxq2_B.svg", transparent=True, format="svg", bbox_inches = "tight")

### Stats

### Read csv created during analysis and create bar plot

In [ ]:
geneList = ['wt','nr2e3']
photoreceptors = ['R','U','S','M','L']

print('{0} vs. {1}:'.format(geneList[0],geneList[1]))

for pr in photoreceptors:
    # get counts for each photoreceptor subtype and exclude NaNs
    wtCount = df[(df['genotype']==(geneList[0]))&(df['excludeFlag']==False)][pr]
    wtCount = wtCount[~np.isnan(wtCount)]
    
    crCount = df[(df['genotype']==(geneList[1]))&(df['excludeFlag']==False)][pr]
    crCount = crCount[~np.isnan(crCount)]
    
    u, p = mannwhitneyu(wtCount, crCount)
    print('\t{0}:'.format(pr))
    print('\t\tU = {0:.3f}, p = {1:.5f}, nEyes: wt = {2:.0f}; cr = {3:.0f}'.format(u,p,len(wtCount),len(crCount)))


In [ ]:
#gene Colors
zfC = {
    'R'  : '#7d7d7d',
    'U' : '#B73AB9',
    'S' : '#4364F6',
    'M' : '#59CB3B',
    'L' : '#CE2A22',
}

zfG = {
    'wt' : '#000000',
    'tbx2a' : '#ab266b',
    'tbx2b' : '#421f8e',
    'foxq2' : '#001dd6',
    'nr2e3' : '#7d7d7d',
}

zfGm = {
    'wt' : 'o',
    'tbx2a' : 'P',
    'tbx2b' : 'X',
    'nr2e3' : '+',
    
}

prLabel = {
    'R'  : 'Rods',
    'U' : 'UV',
    'S' : 'S',
    'M' : 'M',
    'L' : 'L',
}


def formatFigureMain(figH, axH, plotH):
#     font_path = 'C:/Users/pataklk/Documents/Frag_Analysis_Code/Avenir.ttc'
    font_path = '/System/Library/Fonts/Avenir.ttc'
    fontTicks = font_manager.FontProperties(fname=font_path, size=30) # was 18
    fontLabels = font_manager.FontProperties(fname=font_path, size=36) # was 22
    fontTitle = font_manager.FontProperties(fname=font_path, size=30) # was 28 
    axH.set_xscale('linear')
    axH.spines['top'].set_visible(False)
    axH.spines['right'].set_visible(False)
    
    for label in (axH.get_xticklabels() + axH.get_yticklabels()):
        label.set_fontproperties(fontTicks)
    axH.set_xlabel(axH.get_xlabel(), fontproperties = fontTicks)
    axH.set_ylabel(axH.get_ylabel(), fontproperties = fontTicks)
    return fontLabels

def formatFigure(figH, axH, plotH):
    fontLabels = formatFigureMain(figH, axH, plotH)
#     axH.set_xlabel('wt vs. cr', fontproperties=fontLabels)
    axH.set_ylabel('cells per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.xaxis.set_tick_params(rotation=45)

def formatFigureRvU(figH, axH, plotH):
    fontLabels = formatFigureMain(figH, axH, plotH)
    axH.set_xlabel('Rods per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.set_ylabel('UV cones per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.xaxis.set_tick_params(rotation=45)
    
def formatFigureMvS(figH, axH, plotH):
    fontLabels = formatFigureMain(figH, axH, plotH)
    axH.set_xlabel('M cones per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.set_ylabel('S cones per 64 x 64 $\mu$m$^2$', fontproperties=fontLabels)
    axH.xaxis.set_tick_params(rotation=45)

    
def lighten_color(color, amount=0.5):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    import colorsys
    try:
        c = matplotlib.colors.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*matplotlib.colors.to_rgb(c))
    return matplotlib.colors.rgb2hex(colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2]))

def estimateJitter(dataArray):
    """ creates random jitter scaled by local density of points"""
    from scipy.stats import gaussian_kde
    kde = gaussian_kde(dataArray)
    density = kde(dataArray)
    jitter = np.random.randn(len(dataArray))*density
    return jitter
pBGMenu

In [ ]:
#reapply plotting style
applyPlotStyle(pBGMenu.value)

In [ ]:
# Created csv manually during analysis
dPath = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/CRnr2e3F0s/xOGaCT/"
fName = "CRnr2e3F0s_xOGaCT_Counts.csv"

dPath = "/Users/angueyraaristjm/Library/CloudStorage/OneDrive-NationalInstitutesofHealth/zf/Analysis/CRnr2e3F0s/m2Gs2C/"
dPath = "/Users/angueyraaristjm/Documents/eelImaging/tempAnalysis/CRnr2e3F0s/m2Gs2C/"
fName = "CRnr2e3F0s_m2Gs2C_Counts.csv"

df = pd.read_csv(dPath + fName)
df

In [ ]:
geneList = ['wt','nr2e3']
nGenes = np.size(geneList)
# photoreceptors = ['R','U','S','M','L']
photoreceptors = ['S','M']

plotname = ''
fH, axH = plt.subplots(figsize= [12,8])

barW = nGenes+2; # bar width
barD = nGenes+1; # bar distance whitin 1 photoreceptor subtype
barP = -nGenes+1; # position in x-axis
barStep = 0.9; # distance between photoreceptor groups
barPos = []; #array to save bar positions


j=barP;
for gene in geneList:
    i=0;
    j=j+1;
    for pr in photoreceptors:
        i = i+barStep;
        countData = df[(df['genotype']==gene) & (df['excludeFlag']==False)][pr]
        jitter = np.random.randn(len(countData))*0.01
#         textindent = 1/(2.5*barD);
        textindent = 0.10;
        pH = plt.bar([i+j/barD], np.mean(countData), width=1/barW, color=lighten_color(zfC[pr],1), linewidth = 2, edgecolor = lighten_color(zfC[pr],1)); #color=zfC[pr], 
#         pH = plt.bar([i+j/barD], np.mean(countData), yerr = sem(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr][~np.isnan(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr])], ddof = 0), align='center', ecolor='black', capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
#         pH = plt.bar([i+j/barD], np.mean(countData), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr][~np.isnan(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr])], ddof = 0), align='center', ecolor='black', capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
       # pH = plt.bar([i+j/barD], np.mean(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), ecolor=lighten_color(zfC[pr],1.2), capsize=10, width=1/barW, color=zfC[pr], edgecolor = zfC[pr]);
        #pH = plt.errorbar([i+j/barD], np.mean(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), yerr = np.std(df[(df['CRgene']==gene) & (df['Genotype']==geno)][pr]), ecolor=lighten_color(zfC[pr],1.1), elinewidth=2, capsize=5, capthick=2, barsabove= True)
#         pH = plt.scatter(np.ones(len(countData))*[i+j/barD]+jitter, countData, color=lighten_color(zfC[pr],2/3), zorder=2, marker = zfGm[gene], edgecolor='#ffffff', linewidth=0.5, alpha = 0.4);
        pH = plt.errorbar(i+j/barD, np.mean(countData), yerr = [[0],[np.std(countData)]], ecolor=lighten_color(zfC[pr],1), elinewidth=3, capsize=8, capthick=3, zorder=9)
        # plt.text((i+j/barD)-textindent, 3, prLabel[pr], font_properties=font_prop, fontsize=20, ha='left', alpha=0.6)
        pH = plt.scatter(np.ones(len(countData))*[i+j/barD]+jitter, countData, color='#ffffff', zorder=8, marker = 'o', s=50, edgecolor='#ffffff', linewidth=0.5, alpha = .6);
        barPos = np.append(barPos,(i+j/barD))

formatFigure(fH, axH, pH)
axH.set_xticks(np.sort(barPos));

axH.set_xticklabels(geneList * len(photoreceptors));
# axH.set_ylim([0,500]); # this was 400

# savePath = 'C:/Users/pataklk/OneDrive - National Institutes of Health/zf/F0_Analysis/CRfoxq2F0/'
# savePath = "/Users/angueyraaristjm/OneDrive - National Institutes of Health/zf/F0_Analysis/CRfoxq2F0/"
# savePath = "/Users/angueyraaristjm/Documents/LiLab/Presentations/revealjs/resources/20211008_UCLA/"
# plt.savefig(savePath + "Counts_foxq2.png", transparent=True, format="png")
# plt.savefig(savePath + "Counts_foxq2_B.svg", transparent=True, format="svg")